In [4]:
from story_sage.utils.embedding import Embedder, load_chunk_from_disk, embed_documents
from story_sage.story_sage_entity import *
from story_sage.data_classes.story_sage_series import StorySageSeries
import yaml
import glob
import chromadb

series_metadata_name = 'wheel_of_time'

with open('./entities/entities.json', 'r') as f:
    entity_json = json.load(f)
    entity_collection = {key: StorySageEntityCollection.from_dict(value) for key, value in entity_json.items()}

with open('./series_prod.yml', 'r') as f:
    series_dict = yaml.safe_load(f.read())


chroma_client = chromadb.PersistentClient(path='./chroma_data')
embedder = Embedder()
vector_store = chroma_client.get_or_create_collection('story_sage', embedding_function=embedder)


def load_chunks_from_glob(glob_path):
    doc_collection = []
    for chunk_path in glob.glob(glob_path):
        doc_collection.extend(load_chunk_from_disk(chunk_path))
    return doc_collection

doc_collection = load_chunks_from_glob(f'./chunks/{series_metadata_name}/semantic_chunks/*.json')

In [5]:
series_info = [StorySageSeries.from_dict(series) for series in series_dict]

In [7]:
series_id = [series.series_id for series in series_info if series.series_metadata_name == series_metadata_name][0]
print(f'Processing series {series_metadata_name} with id {series_id}')

Processing series wheel_of_time with id 3


In [8]:
embed_documents(doc_collection=doc_collection, vector_store=vector_store, series_id=series_id, entity_collection=entity_collection[series_metadata_name])

Identifying characters in documents:   0%|          | 0/21896 [00:00<?, ?it/s]

Upserting documents in batches:   0%|          | 0/22 [00:00<?, ?it/s]